In [1]:
import collections
import json
import pathlib
import shutil
import zipfile

In [2]:
jsons = pathlib.Path.cwd() / 'data' / 'jsons'
js = list(p for p in jsons.iterdir() if p.name.endswith('json'))
list(enumerate(js))

[(0,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2013-01-01__2013-12-31.json')),
 (1,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2014-01-01__2014-12-31.json')),
 (2,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2016-01-01__2016-12-31.json')),
 (3,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2015-01-01__2015-12-31.json')),
 (4,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2014-02-09__2014-12-31.json')),
 (5,
  PosixPath('/Users/matteding/Desktop/Summarizer/data/jsons/2013-06-16__2013-12-31.json'))]

In [16]:
file = js[0]

with open(file) as fp:
    dicts = []
    for line in fp.readlines():
        dct = json.loads(line)
        dicts.append(dct)
        
with open(file) as fp:
    lines = fp.readlines()

In [11]:
has_story = [d for d in dicts if d['story']]
no_story = [d for d in dicts if not d['story']] # one known was simply a map; see Jan/Feb 2010
len(has_story), len(no_story)

(1917, 0)

In [9]:
counter = collections.Counter(lines)
dupes = [json.loads(k) for k, v in counter.items() if v > 1]
dupes = [(j['date'], j['title']) for j in dupes]
len(counter), len(dupes)

(1841, 76)

### Merge Year

In [13]:
year = str(2018)
unique = set()

for path in (j for j in (jsons / year).iterdir() if j.name.endswith('json') and '_' in j.name):
    with open(path) as fp:
        unique.update(fp.readlines())
        
dicts = [json.loads(article) for article in unique]
merged = {year: [d for d in dicts if d['story']]}
filepath = jsons / year / f'{year}.json'
with open(filepath, 'w') as fp:
    json.dump(merged, fp)

len(merged[year])

5207

### Compress Year

In [14]:
file_to_zip = f'{year}.json'
target_path = jsons / year

compression = 'xztar'
try:
    shutil.make_archive(target_path / year, compression, target_path, file_to_zip)
except OSError:
    raise

### Null-Story

In [30]:
[d for d in dicts if not d.get('story')]

[{'date': 'March 2, 2010 ',
  'title': 'Interactive Map: U.S. Rates Of Child Abuse Fatalities',
  'author': None,
  'summary': ' Using data from a report by the Department of Health and Human Services, Administration for Children & Families, this map chronicles child fatalities by maltreatment in most states. However, state data on child fatality from maltreatment is difficult to compare: There is no federal definition of child abuse, so cases are interpreted differently.',
  'story': ''}]